# Threshold images

K.Garner, 2020

This wf takes an spmT image, anatomically masks it, and then applies a threshold to retain the top 10%
of t-values within each ROI. The remaining is saved.
Designed to be repeated over t's and over masks.

In [1]:
from nipype.interfaces.io import BIDSDataGrabber, DataFinder, DataSink, DataGrabber
import nipype.pipeline as pe
import nipype as ni
import nipype.interfaces.fsl.maths as fsl
from nipype.interfaces import spm as spm
from nipype.algorithms.misc import Gunzip 
import pandas as pd
import os, re, json
# https://nipype.readthedocs.io/en/0.11.0/users/spmmcr.html

matlab_cmd = '/opt/spm12-r7219/run_spm12.sh /opt/matlabmcr-2010a/v713/ script'
spm.SPMCommand.set_mlab_paths(matlab_cmd=matlab_cmd, use_mcr=True)

201214-11:21:23,46 nipype.utils WARNING:
	 A newer version (1.5.1) of nipy/nipype is available. You are using 1.5.0


In [2]:
Basedir = "/scratch/qbi/uqkgarn1/STRIWP1/"
thrsh = pe.Workflow(name="thrsh") # workflow to run the analysis

## Data grabbing and sinking

In [3]:
dg = pe.Node(DataGrabber(infields=['sub', 'TR'], 
                         outfields=['ts','msks']), name='data-grabber')
dg.inputs.base_dir = "/scratch/qbi/uqkgarn1/STRIWP1/"
dg.inputs.sort_filelist = True
dg.inputs.template='*'
dg.inputs.template_args = {'ts': [['sub', 'TR']],
                           'msks': [['sub', 'TR']]}
dg.inputs.field_template = {'msks': '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-%s/TR%s/MASKS/*.nii.gz',
                            'ts': '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-%s/TR%s/FLGLM/spmT*.nii.gz'}

In [4]:
# debugging
dg.inputs.sub = '03'
dg.inputs.TR = '700'
res = dg.run()
res.outputs

201214-11:21:25,277 nipype.workflow INFO:
	 [Node] Setting-up "data-grabber" in "/tmp/tmpqy13b9_2/data-grabber".
201214-11:21:25,282 nipype.workflow INFO:
	 [Node] Running "data-grabber" ("nipype.interfaces.io.DataGrabber")
201214-11:21:25,289 nipype.workflow INFO:
	 [Node] Finished "data-grabber".



msks = ['/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-03/TR700/MASKS/rCN_trans.nii.gz', '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-03/TR700/MASKS/rFEF_trans.nii.gz', '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-03/TR700/MASKS/rGPe_trans.nii.gz', '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-03/TR700/MASKS/rGPi_trans.nii.gz', '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-03/TR700/MASKS/rIPS_trans.nii.gz', '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-03/TR700/MASKS/rLOC_trans.nii.gz', '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-03/TR700/MASKS/rPut_trans.nii.gz', '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-03/TR700/MASKS/rSTN_trans.nii.gz', '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-03/TR700/MASKS/rVS_trans.nii.gz']
ts = ['/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-03/TR700/FLGLM/spmT_0001.nii.gz', '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-03/TR700/FLGLM/spmT_0002.nii.gz', '/scratch/qbi/uqkgarn1/STRIWP1/

In [5]:
def printSubPath(fullFilePath):
    # function to split filepath into constituent parts, then print string to add as input to DataSink for the container string
    # given the full filepath, this extracts the subject folder and TR strings for input
    # into DataSink
    import os
    import re
    fname = os.path.normpath(fullFilePath[0])
    l = fname.split(os.sep)
    name = [s for s in l if re.search('sub', s)][0], [t for t in l if re.search('TR([0-9])', t)][0]
    name = '/'.join(name)
    name
    return name

In [6]:
#printSubPath(res.outputs.msks)

In [7]:
ds = pe.Node(DataSink(), name='sink-stuff')
ds.inputs.base_directory = "/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/"
substitutions = [('_TR_([0-9]*)_sub_([0-9]*)', ''),
                 ('_thrsh-t([0-9]*)', ''),
                 ('_absthat([0-9]*)', ''),
                 ('_mask-t([0-9])*', ''),
                 ('_multi([0-9])*', '')] 
ds.inputs.regexp_substitutions = substitutions

In [8]:
datinf = pe.Node(ni.IdentityInterface(fields=['sub', 'TR']),
                 name='dat-info')
#datinf.iterables = [('sub', ['01', '02', '03', '04', '05']), ('TR', ['700', '1510', '1920'])]
datinf.iterables = [('sub', ['02', '03']), ('TR', ['1510'])]
datinf.iterables 

[('sub', ['02', '03']), ('TR', ['1510'])]

## Gzip interface

get .nii -> .nii.gz

In [9]:
from nipype.interfaces.base import (
    TraitedSpec,
    CommandLineInputSpec,
    CommandLine,
    File
)
import os

class GZipInputSpec(CommandLineInputSpec):
    in_file = File(desc="File", exists=True, mandatory=True, argstr="%s")

class GZipOutputSpec(TraitedSpec):
    out_file = File(desc = "Zip file", exists = True)

class GZipTask(CommandLine):
    input_spec = GZipInputSpec
    output_spec = GZipOutputSpec
    _cmd = 'gzip'

    def _list_outputs(self):
            outputs = self.output_spec().get()
            outputs['out_file'] = os.path.abspath(self.inputs.in_file + ".gz")
            return outputs
        
gzip = pe.MapNode(GZipTask(), iterfield='in_file', name='gzip')

## functions for list cats

In [10]:
def tcat(fullFileList, nMasks=9):
    return fullFileList*nMasks

In [11]:
tcat(res.outputs.ts)

['/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-03/TR700/FLGLM/spmT_0001.nii.gz',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-03/TR700/FLGLM/spmT_0002.nii.gz',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-03/TR700/FLGLM/spmT_0003.nii.gz',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-03/TR700/FLGLM/spmT_0005.nii.gz',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-03/TR700/FLGLM/spmT_0006.nii.gz',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-03/TR700/FLGLM/spmT_0007.nii.gz',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-03/TR700/FLGLM/spmT_0009.nii.gz',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-03/TR700/FLGLM/spmT_0001.nii.gz',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-03/TR700/FLGLM/spmT_0002.nii.gz',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-03/TR700/FLGLM/spmT_0003.nii.gz',
 '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-03/TR700/FLGLM/spmT_00

In [12]:
def maskcat(fullFileList, nCon=7):
    import numpy as np
    return np.ndarray.tolist(np.repeat(fullFileList, nCon))

In [13]:
#maskcat(res.outputs.msks)

## define output names for t x msk

In [14]:
tnms = ['spmT_0001', 'spmT_0002', 'spmT_0003', 'spmT_0005', 'spmT_0006', 'spmT_0007', 'spmT_0009']
msks = ['rCN', 'rFEF', 'rGPe', 'rGPi', 'rIPS', 'rLOC', 'rPut', 'rSTN', 'rVS']
touts = ['th'+i+'_'+n+'.nii.gz' for i in msks for n in tnms]

In [15]:
touts

['thrCN_spmT_0001.nii.gz',
 'thrCN_spmT_0002.nii.gz',
 'thrCN_spmT_0003.nii.gz',
 'thrCN_spmT_0005.nii.gz',
 'thrCN_spmT_0006.nii.gz',
 'thrCN_spmT_0007.nii.gz',
 'thrCN_spmT_0009.nii.gz',
 'thrFEF_spmT_0001.nii.gz',
 'thrFEF_spmT_0002.nii.gz',
 'thrFEF_spmT_0003.nii.gz',
 'thrFEF_spmT_0005.nii.gz',
 'thrFEF_spmT_0006.nii.gz',
 'thrFEF_spmT_0007.nii.gz',
 'thrFEF_spmT_0009.nii.gz',
 'thrGPe_spmT_0001.nii.gz',
 'thrGPe_spmT_0002.nii.gz',
 'thrGPe_spmT_0003.nii.gz',
 'thrGPe_spmT_0005.nii.gz',
 'thrGPe_spmT_0006.nii.gz',
 'thrGPe_spmT_0007.nii.gz',
 'thrGPe_spmT_0009.nii.gz',
 'thrGPi_spmT_0001.nii.gz',
 'thrGPi_spmT_0002.nii.gz',
 'thrGPi_spmT_0003.nii.gz',
 'thrGPi_spmT_0005.nii.gz',
 'thrGPi_spmT_0006.nii.gz',
 'thrGPi_spmT_0007.nii.gz',
 'thrGPi_spmT_0009.nii.gz',
 'thrIPS_spmT_0001.nii.gz',
 'thrIPS_spmT_0002.nii.gz',
 'thrIPS_spmT_0003.nii.gz',
 'thrIPS_spmT_0005.nii.gz',
 'thrIPS_spmT_0006.nii.gz',
 'thrIPS_spmT_0007.nii.gz',
 'thrIPS_spmT_0009.nii.gz',
 'thrLOC_spmT_0001.nii.gz',

## FSL functionality to apply anatomical mask

In [16]:
# maskt = pe.MapNode(fsl.ApplyMask(), iterfield = ['in_file', 'mask_file', 'out_file'], name='mask-t')
# maskt.inputs.out_file = touts
# maskt.inputs.nan2zeros = True

maskt = pe.MapNode(fsl.BinaryMaths(), iterfield = ['in_file', 'operand_file'], name='mask-t')
maskt.inputs.operation = 'mul'
maskt.inputs.nan2zeros = True

## to take the absolute value of the image

In [17]:
absv = pe.MapNode(fsl.UnaryMaths(), iterfield = ['in_file'], name='absthat')
absv.inputs.operation = 'abs'

# rounding test

In [18]:
# multIm = pe.MapNode(fsl.BinaryMaths(), iterfield = ['in_file', 'out_file'], name='multi')
# multIm.inputs.operand_value = 10
# multIm.inputs.operation = 'mul'
# multIm.inputs.out_file = touts

## and to apply threshold

In [19]:
thrsht = pe.MapNode(fsl.Threshold(), iterfield = ['in_file', 'out_file'], name='thrsh-t')
thrsht.inputs.thresh = 2.326 # p=.01 - 2 tailed
thrsht.inputs.out_file = touts
#thrsht.inputs.internal_datatype = 'input'
# thrsht.inputs.use_robust_range = True
# thrsht.inputs.use_nonzero_voxels = True

## Connect workflow

In [20]:
# thrsh.connect([(datinf, dg, [('sub', 'sub')]),
#               (datinf, dg, [('TR', 'TR')]),
#               (dg, ds, [(('msks', printSubPath), 'container')]),
# #               (dg, gzip, [('ts', 'in_file')]), # zip the t-images # use these 2 lines if .nii -> .nii.gz is required
# #               (gzip, maskt, [(('out_file', tcat), 'in_file')]),
#               (dg, maskt, [(('ts', tcat), 'in_file')]), 
#               (dg, maskt, [(('msks', maskcat), 'mask_file')]),
#               (maskt, absv, [('out_file', 'in_file')]), 
#               (absv, thrsht, [('out_file', 'in_file')]),
#               (thrsht, ds, [('out_file', 'THRSH')])
# ])
# res = thrsh.run()

## Debugging workflow

In [21]:
thrsh.connect([(datinf, dg, [('sub', 'sub')]),
               (datinf, dg, [('TR', 'TR')]),
               (dg, ds, [(('msks', printSubPath), 'container')]),
#               (dg, gzip, [('ts', 'in_file')]), # zip the t-images # use these 2 lines if .nii -> .nii.gz is required
#               (gzip, maskt, [(('out_file', tcat), 'in_file')]),
               (dg, maskt, [(('ts', tcat), 'in_file')]), 
               (dg, maskt, [(('msks', maskcat), 'operand_file')]),
               (maskt, absv, [('out_file', 'in_file')]),
               (absv, thrsht, [('out_file', 'in_file')]),
               (thrsht, ds, [('out_file', 'THRSH')])
])
res = thrsh.run()

201214-11:21:25,417 nipype.workflow INFO:
	 Workflow thrsh settings: ['check', 'execution', 'logging', 'monitoring']
201214-11:21:25,438 nipype.workflow INFO:
	 Running serially.
201214-11:21:25,439 nipype.workflow INFO:
	 [Node] Setting-up "thrsh.data-grabber" in "/tmp/tmpqy13b9_2/data-grabber".
201214-11:21:25,444 nipype.workflow INFO:
	 [Node] Running "data-grabber" ("nipype.interfaces.io.DataGrabber")
201214-11:21:25,449 nipype.workflow INFO:
	 [Node] Finished "thrsh.data-grabber".
201214-11:21:25,449 nipype.workflow INFO:
	 [Node] Setting-up "thrsh.mask-t" in "/tmp/tmpow36okow/thrsh/_TR_1510_sub_03/mask-t".
201214-11:21:25,469 nipype.workflow INFO:
	 [Node] Setting-up "_mask-t0" in "/tmp/tmpow36okow/thrsh/_TR_1510_sub_03/mask-t/mapflow/_mask-t0".
201214-11:21:25,474 nipype.workflow INFO:
	 [Node] Running "_mask-t0" ("nipype.interfaces.fsl.maths.BinaryMaths"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-03/TR1510/FLGLM/spmT

201214-11:21:28,464 nipype.workflow INFO:
	 [Node] Finished "_mask-t11".
201214-11:21:28,466 nipype.workflow INFO:
	 [Node] Setting-up "_mask-t12" in "/tmp/tmpow36okow/thrsh/_TR_1510_sub_03/mask-t/mapflow/_mask-t12".
201214-11:21:28,471 nipype.workflow INFO:
	 [Node] Running "_mask-t12" ("nipype.interfaces.fsl.maths.BinaryMaths"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-03/TR1510/FLGLM/spmT_0007.nii.gz -nan -mul /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-03/TR1510/MASKS/rFEF_trans.nii.gz /tmp/tmpow36okow/thrsh/_TR_1510_sub_03/mask-t/mapflow/_mask-t12/spmT_0007_maths.nii.gz
201214-11:21:28,716 nipype.workflow INFO:
	 [Node] Finished "_mask-t12".
201214-11:21:28,718 nipype.workflow INFO:
	 [Node] Setting-up "_mask-t13" in "/tmp/tmpow36okow/thrsh/_TR_1510_sub_03/mask-t/mapflow/_mask-t13".
201214-11:21:28,723 nipype.workflow INFO:
	 [Node] Running "_mask-t13" ("nipype.interfaces.fsl.maths.BinaryMaths"), a CommandLine Int

201214-11:21:31,711 nipype.workflow INFO:
	 [Node] Finished "_mask-t24".
201214-11:21:31,713 nipype.workflow INFO:
	 [Node] Setting-up "_mask-t25" in "/tmp/tmpow36okow/thrsh/_TR_1510_sub_03/mask-t/mapflow/_mask-t25".
201214-11:21:31,717 nipype.workflow INFO:
	 [Node] Running "_mask-t25" ("nipype.interfaces.fsl.maths.BinaryMaths"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-03/TR1510/FLGLM/spmT_0006.nii.gz -nan -mul /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-03/TR1510/MASKS/rGPi_trans.nii.gz /tmp/tmpow36okow/thrsh/_TR_1510_sub_03/mask-t/mapflow/_mask-t25/spmT_0006_maths.nii.gz
201214-11:21:31,967 nipype.workflow INFO:
	 [Node] Finished "_mask-t25".
201214-11:21:31,969 nipype.workflow INFO:
	 [Node] Setting-up "_mask-t26" in "/tmp/tmpow36okow/thrsh/_TR_1510_sub_03/mask-t/mapflow/_mask-t26".
201214-11:21:31,973 nipype.workflow INFO:
	 [Node] Running "_mask-t26" ("nipype.interfaces.fsl.maths.BinaryMaths"), a CommandLine Int

201214-11:21:35,208 nipype.workflow INFO:
	 [Node] Finished "_mask-t37".
201214-11:21:35,210 nipype.workflow INFO:
	 [Node] Setting-up "_mask-t38" in "/tmp/tmpow36okow/thrsh/_TR_1510_sub_03/mask-t/mapflow/_mask-t38".
201214-11:21:35,215 nipype.workflow INFO:
	 [Node] Running "_mask-t38" ("nipype.interfaces.fsl.maths.BinaryMaths"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-03/TR1510/FLGLM/spmT_0005.nii.gz -nan -mul /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-03/TR1510/MASKS/rLOC_trans.nii.gz /tmp/tmpow36okow/thrsh/_TR_1510_sub_03/mask-t/mapflow/_mask-t38/spmT_0005_maths.nii.gz
201214-11:21:35,476 nipype.workflow INFO:
	 [Node] Finished "_mask-t38".
201214-11:21:35,478 nipype.workflow INFO:
	 [Node] Setting-up "_mask-t39" in "/tmp/tmpow36okow/thrsh/_TR_1510_sub_03/mask-t/mapflow/_mask-t39".
201214-11:21:35,483 nipype.workflow INFO:
	 [Node] Running "_mask-t39" ("nipype.interfaces.fsl.maths.BinaryMaths"), a CommandLine Int

201214-11:21:38,560 nipype.workflow INFO:
	 [Node] Finished "_mask-t50".
201214-11:21:38,563 nipype.workflow INFO:
	 [Node] Setting-up "_mask-t51" in "/tmp/tmpow36okow/thrsh/_TR_1510_sub_03/mask-t/mapflow/_mask-t51".
201214-11:21:38,568 nipype.workflow INFO:
	 [Node] Running "_mask-t51" ("nipype.interfaces.fsl.maths.BinaryMaths"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-03/TR1510/FLGLM/spmT_0003.nii.gz -nan -mul /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-03/TR1510/MASKS/rSTN_trans.nii.gz /tmp/tmpow36okow/thrsh/_TR_1510_sub_03/mask-t/mapflow/_mask-t51/spmT_0003_maths.nii.gz
201214-11:21:38,841 nipype.workflow INFO:
	 [Node] Finished "_mask-t51".
201214-11:21:38,844 nipype.workflow INFO:
	 [Node] Setting-up "_mask-t52" in "/tmp/tmpow36okow/thrsh/_TR_1510_sub_03/mask-t/mapflow/_mask-t52".
201214-11:21:38,849 nipype.workflow INFO:
	 [Node] Running "_mask-t52" ("nipype.interfaces.fsl.maths.BinaryMaths"), a CommandLine Int

...

201214-11:22:35,433 nipype.workflow INFO:
	 [Node] Finished "_thrsh-t23".
201214-11:22:35,435 nipype.workflow INFO:
	 [Node] Setting-up "_thrsh-t24" in "/tmp/tmp9ydcq8e5/thrsh/_TR_1510_sub_02/thrsh-t/mapflow/_thrsh-t24".
201214-11:22:35,440 nipype.workflow INFO:
	 [Node] Running "_thrsh-t24" ("nipype.interfaces.fsl.maths.Threshold"), a CommandLine Interface with command:
fslmaths /tmp/tmpn7urikt7/thrsh/_TR_1510_sub_02/absthat/mapflow/_absthat24/spmT_0005_maths_abs.nii.gz -thr 2.3260000000 thrGPi_spmT_0005.nii.gz
201214-11:22:35,586 nipype.workflow INFO:
	 [Node] Finished "_thrsh-t24".
201214-11:22:35,589 nipype.workflow INFO:
	 [Node] Setting-up "_thrsh-t25" in "/tmp/tmp9ydcq8e5/thrsh/_TR_1510_sub_02/thrsh-t/mapflow/_thrsh-t25".
201214-11:22:35,593 nipype.workflow INFO:
	 [Node] Running "_thrsh-t25" ("nipype.interfaces.fsl.maths.Threshold"), a CommandLine Interface with command:
fslmaths /tmp/tmpn7urikt7/thrsh/_TR_1510_sub_02/absthat/mapflow/_absthat25/spmT_0006_maths_abs.nii.gz -thr 2

201214-11:22:38,74 nipype.workflow INFO:
	 [Node] Finished "_thrsh-t39".
201214-11:22:38,76 nipype.workflow INFO:
	 [Node] Setting-up "_thrsh-t40" in "/tmp/tmp9ydcq8e5/thrsh/_TR_1510_sub_02/thrsh-t/mapflow/_thrsh-t40".
201214-11:22:38,80 nipype.workflow INFO:
	 [Node] Running "_thrsh-t40" ("nipype.interfaces.fsl.maths.Threshold"), a CommandLine Interface with command:
fslmaths /tmp/tmpn7urikt7/thrsh/_TR_1510_sub_02/absthat/mapflow/_absthat40/spmT_0007_maths_abs.nii.gz -thr 2.3260000000 thrLOC_spmT_0007.nii.gz
201214-11:22:38,233 nipype.workflow INFO:
	 [Node] Finished "_thrsh-t40".
201214-11:22:38,235 nipype.workflow INFO:
	 [Node] Setting-up "_thrsh-t41" in "/tmp/tmp9ydcq8e5/thrsh/_TR_1510_sub_02/thrsh-t/mapflow/_thrsh-t41".
201214-11:22:38,239 nipype.workflow INFO:
	 [Node] Running "_thrsh-t41" ("nipype.interfaces.fsl.maths.Threshold"), a CommandLine Interface with command:
fslmaths /tmp/tmpn7urikt7/thrsh/_TR_1510_sub_02/absthat/mapflow/_absthat41/spmT_0009_maths_abs.nii.gz -thr 2.32

201214-11:22:40,612 nipype.workflow INFO:
	 [Node] Finished "_thrsh-t55".
201214-11:22:40,615 nipype.workflow INFO:
	 [Node] Setting-up "_thrsh-t56" in "/tmp/tmp9ydcq8e5/thrsh/_TR_1510_sub_02/thrsh-t/mapflow/_thrsh-t56".
201214-11:22:40,619 nipype.workflow INFO:
	 [Node] Running "_thrsh-t56" ("nipype.interfaces.fsl.maths.Threshold"), a CommandLine Interface with command:
fslmaths /tmp/tmpn7urikt7/thrsh/_TR_1510_sub_02/absthat/mapflow/_absthat56/spmT_0001_maths_abs.nii.gz -thr 2.3260000000 thrVS_spmT_0001.nii.gz
201214-11:22:40,794 nipype.workflow INFO:
	 [Node] Finished "_thrsh-t56".
201214-11:22:40,796 nipype.workflow INFO:
	 [Node] Setting-up "_thrsh-t57" in "/tmp/tmp9ydcq8e5/thrsh/_TR_1510_sub_02/thrsh-t/mapflow/_thrsh-t57".
201214-11:22:40,801 nipype.workflow INFO:
	 [Node] Running "_thrsh-t57" ("nipype.interfaces.fsl.maths.Threshold"), a CommandLine Interface with command:
fslmaths /tmp/tmpn7urikt7/thrsh/_TR_1510_sub_02/absthat/mapflow/_absthat57/spmT_0002_maths_abs.nii.gz -thr 2.

201214-11:22:41,874 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-02/TR1510/THRSH/_TR_1510_sub_02/_thrsh-t16/thrGPe_spmT_0003.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-02/TR1510/THRSH///thrGPe_spmT_0003.nii.gz
201214-11:22:41,876 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-02/TR1510/THRSH/_TR_1510_sub_02/_thrsh-t17/thrGPe_spmT_0005.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-02/TR1510/THRSH///thrGPe_spmT_0005.nii.gz
201214-11:22:41,877 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-02/TR1510/THRSH/_TR_1510_sub_02/_thrsh-t18/thrGPe_spmT_0006.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-02/TR1510/THRSH///thrGPe_spmT_0006.nii.gz
201214-11:22:41,879 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-02/TR1510/THRSH/_TR_1510_sub_02/_thrsh-t19/thrGPe_spmT_0007.nii.gz -> /s

201214-11:22:41,928 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-02/TR1510/THRSH/_TR_1510_sub_02/_thrsh-t46/thrPut_spmT_0006.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-02/TR1510/THRSH///thrPut_spmT_0006.nii.gz
201214-11:22:41,930 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-02/TR1510/THRSH/_TR_1510_sub_02/_thrsh-t47/thrPut_spmT_0007.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-02/TR1510/THRSH///thrPut_spmT_0007.nii.gz
201214-11:22:41,932 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-02/TR1510/THRSH/_TR_1510_sub_02/_thrsh-t48/thrPut_spmT_0009.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-02/TR1510/THRSH///thrPut_spmT_0009.nii.gz
201214-11:22:41,934 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-02/TR1510/THRSH/_TR_1510_sub_02/_thrsh-t49/thrSTN_spmT_0001.nii.gz -> /s